In [6]:
import psycopg2
import pandas as pd

In [10]:
conn = psycopg2.connect(
    host="localhost",
    database="Restaurant_info",
    user="postgres",
    password="root@123"
)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the query parameters
store_id = 4430370444829587327	
start_timestamp = '2023-01-25 20:00:00'
end_timestamp = '2023-01-25 20:59:59'

# Construct the SQL query
query = """
    SELECT *
    FROM recent_status
    WHERE store_id = %s
      AND timestamp_utc >= %s
      AND timestamp_utc <= %s;
"""

# Execute the query with the provided parameters
cursor.execute(query, (store_id, start_timestamp, end_timestamp))

# Fetch all the rows from the query result
rows = cursor.fetchall()

In [34]:
columns = ["Store_id", "Status","TimeStamp"]
queries = pd.DataFrame(rows,columns=columns)
queries

,Store_id,Status,TimeStamp
0,4430370444829587327,active,2023-01-25 20:31:12.481962+05:30


In [32]:
queries['TimeStamp']

0   2023-01-25 20:31:12.481962+05:30
Name: TimeStamp, dtype: datetime64[ns, UTC+05:30]

In [50]:
safety = pd.read_csv('utc active times.csv')
safety

,store_id,day,start_time_local,end_time_local
0,1481966498820158979,4,2023-06-04 04:00:00,2023-06-04 04:10:00
1,1481966498820158979,2,2023-06-04 04:00:00,2023-06-04 04:10:00
2,1481966498820158979,0,2023-06-04 04:00:00,2023-06-04 04:10:00
3,1481966498820158979,1,2023-06-04 04:00:00,2023-06-04 04:10:00
4,1481966498820158979,5,2023-06-04 04:00:00,2023-06-04 04:10:00
...,...,...,...,...
86193,1774428314218854399,4,2023-06-04 04:00:00,2023-06-05 03:59:59
86194,1774428314218854399,6,2023-06-04 04:00:00,2023-06-05 03:59:59
86195,1774428314218854399,3,2023-06-04 04:00:00,2023-06-05 03:59:59
86196,1774428314218854399,2,2023-06-04 04:00:00,2023-06-05 03:59:59


In [51]:
df = safety
def calculate_active_time(store_id, day, current_time):
    # Filter the dataframe based on store ID and day
    filtered_df = df[(df['store_id'] == store_id) & (df['day'] == day)]
    
    if filtered_df.empty:
        return 0, 0, 0
    
    start_times = filtered_df['start_time_local']
    end_times = filtered_df['end_time_local']
    
    # Calculate active time in the last hour
    last_hour_active_time = sum((current_time - start_times <= pd.Timedelta(hours=1)) & (end_times >= current_time))
    
    # Calculate active time in the last day
    last_day_active_time = sum((current_time - start_times <= pd.Timedelta(days=1)) & (end_times >= current_time))
    
    # Calculate active time in the last week
    last_week_active_time = sum((current_time - start_times <= pd.Timedelta(weeks=1)) & (end_times >= current_time))
    
    return last_hour_active_time, last_day_active_time, last_week_active_time



In [52]:
calculate_active_time(1481966498820158979,4,"2023-06-04 04:09:00")

TypeError: unsupported operand type(s) for -: 'str' and 'str'

True